# Depictio Component Export API Demo

This notebook demonstrates how to **export rendered dashboard components** from Depictio
and reconstruct them in a standalone Python environment — no Depictio UI required.

We use the `/render` API endpoint to fetch:
1. **Figure** — a Plotly scatter plot, rendered as a standalone interactive figure
2. **Card** — a computed aggregation value (average sepal length)
3. **Table** — raw tabular data loaded into a pandas DataFrame

All examples target the **Iris Dashboard demo** (`6824cb3b89d2b72169309737`).

In [ ]:
import requests

API_BASE = "http://localhost:8135/depictio/api/v1"
DASHBOARD_ID = "6824cb3b89d2b72169309737"
TOKEN = (
    "eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCJ9."
    "eyJuYW1lIjoiZGVmYXVsdF90b2tlbiIsInRva2VuX2xpZmV0aW1lIjoibG9uZy1saXZlZCIsInRva2VuX3R5cGUiOiJiZWFyZXIiLCJzdWIiOiI2NzY1OGJhMDMzYzhiNTlhZDQ4OWQ3YzciLCJleHAiOjE4MDM3MjczMjR9."
    "kXyOArNVvh9UMjcNRchdd6bhbV6TivUskevm-1X7IZw2UtSjdysxq4V7e0w4NC2CRtPoX7fiSxoRh172-X8J0fEc1wlX8432zBq1o64HMExED9vUku1ubcamxDIq3QVaVo25pq4qiVWpwhMVhf3TI5O7h1ZJCXflpyWTh11-0rZ5TeObf-i1pJ6Z-Wa2p4pPf5MkHufm2U0iFL6Ce3QaXqX4dA-LxZov1FSR5rKru7axHL_0pTk9uTyoBLsMTN7uG6jxUOBQfb29yy9zob5f8gS9MAfgd64A7UvOGi0_hXs_sQirf_2YRrA30k_0Yz2ygpRD9JJJHNHVkqnFBRFzOA"
)

HEADERS = {"Authorization": f"Bearer {TOKEN}", "Content-Type": "application/json"}


def render_component(identifier: str, theme: str = "light", timeout: int = 30, **kwargs) -> dict:
    """Call the Depictio render endpoint and return the JSON response.
    
    identifier can be a component UUID (index) or a human-readable tag.
    """
    url = f"{API_BASE}/render/{DASHBOARD_ID}/components/{identifier}"
    payload = {"theme": theme, "timeout": timeout, **kwargs}
    resp = requests.post(url, json=payload, headers=HEADERS)
    resp.raise_for_status()
    return resp.json()

In [ ]:
# Quick check: is the /render endpoint registered?
# OpenAPI schema is at the app root, not under the API prefix
base_url = API_BASE.split("/depictio/")[0]  # http://localhost:8135
resp = requests.get(f"{base_url}/openapi.json", headers=HEADERS)
if resp.ok:
    paths = [p for p in resp.json().get("paths", {}) if "/render/" in p]
    if paths:
        print("Render routes registered:")
        for p in paths:
            print(f"  {p}")
    else:
        print("WARNING: No /render/ routes found in OpenAPI schema!")
        print("The render router may have failed to load (check API logs for import errors).")
else:
    print(f"Could not fetch OpenAPI schema: {resp.status_code}")

---
## Discover available components

Fetch the dashboard via the GET endpoint and list all component identifiers (UUID index + tag).

In [ ]:
# List all components using the render list endpoint (returns index, tag, type, renderable flag)
resp = requests.get(f"{API_BASE}/render/{DASHBOARD_ID}/components", headers=HEADERS)
resp.raise_for_status()
components = resp.json()

print(f"{'Type':<15} {'Renderable':<12} {'Tag':<50} {'Index (UUID)'}")
print("-" * 120)
for comp in components:
    print(
        f"{comp['component_type']:<15} "
        f"{'yes' if comp['renderable'] else 'no':<12} "
        f"{comp['tag']:<50} "
        f"{comp['index']}"
    )

# Build lookup dicts by component type for easy access below
figures = [c for c in components if c["component_type"] == "figure"]
cards = [c for c in components if c["component_type"] == "card"]
tables = [c for c in components if c["component_type"] == "table"]
print(f"\nRenderable: {len(figures)} figures, {len(cards)} cards, {len(tables)} tables")

---
## Example 1: Export a Plotly Figure

Render the **scatter plot** (sepal length vs. sepal width, colored by variety) and
display it as a standalone Plotly figure — identical to what Depictio shows, but
running entirely outside the platform.

In [ ]:
import plotly.graph_objects as go

# Render the first figure component (use its tag from discovery above)
figure_tag = figures[0]["tag"]
print(f"Rendering figure: {figure_tag}")

result = render_component(figure_tag, theme="light")
print(f"Status: {result['status']}")
print(f"Component type: {result['component_type']}")
print(f"Component tag: {result['component_tag']}")

# Reconstruct the Plotly figure from the exported JSON
fig = go.Figure(result["data"]["figure"])
fig.update_layout(title_text="Exported from Depictio")
fig.show()

In [ ]:
# The data_info field tells us about sampling and row counts
print("Data info:", result["data"].get("data_info", {}))

---
## Example 2: Export a Card (Computed Aggregation)

Render the **average sepal length** card. This returns a single computed value
that can be used in reports, alerts, or downstream pipelines.

In [ ]:
# Render the first card component
card_tag = cards[0]["tag"]
print(f"Rendering card: {card_tag}")

result = render_component(card_tag)
print(f"Status: {result['status']}")
print(f"Component type: {result['component_type']}")
print(f"Component tag: {result['component_tag']}")

card = result["data"]
print(f"\n--- Exported Card ---")
print(f"Column:      {card['column']}")
print(f"Aggregation: {card['aggregation']}")
print(f"Value:       {card['value']}")

In [ ]:
# Use the value programmatically — e.g. in a threshold check
threshold = 6.0
avg = card["value"]
status = "ABOVE" if avg >= threshold else "BELOW"
print(f"Average sepal length ({avg:.2f}) is {status} threshold ({threshold})")

---
## Example 3: Export a Table as a DataFrame

Render the **data table** component and load it directly into pandas for
analysis, joining with other data, or exporting to CSV/Excel.

In [ ]:
import pandas as pd

# Render the first table component
table_tag = tables[0]["tag"]
print(f"Rendering table: {table_tag}")

result = render_component(table_tag)
print(f"Status: {result['status']}")
print(f"Component type: {result['component_type']}")
print(f"Total rows in source: {result['data']['total_rows']}")
print(f"Rows returned:        {result['data']['returned_rows']}")

# Load into pandas
df = pd.DataFrame(result["data"]["rows"], columns=result["data"]["columns"])
df.head(10)

In [ ]:
# Quick analysis on the exported data
print(df.describe())
print(f"\nVarieties: {df['variety'].unique().tolist() if 'variety' in df.columns else 'N/A'}")

In [ ]:
# Export to CSV — the data now lives completely outside Depictio
df.to_csv("iris_exported.csv", index=False)
print("Saved to iris_exported.csv")

---
## Summary

The render API has two endpoints:

| Endpoint | Method | Purpose |
|----------|--------|---------|
| `/render/{dashboard_id}/components` | GET | List all components with their `tag`, `index`, and type |
| `/render/{dashboard_id}/components/{identifier}` | POST | Render a component by tag or UUID index |

Components can be identified by **tag** (human-readable, e.g. `figure-scatter_sepal_length-a3f4d2`)
or **index** (UUID, e.g. `1e9febfc-e48c-4614-8f0d-5c09ba73991f`).

| Component type | What you get |
|----------------|-------------|
| **Figure** | Full Plotly JSON — `go.Figure(data["figure"])` |
| **Card** | Scalar value + aggregation metadata |
| **Table** | Row data + column list — `pd.DataFrame(data["rows"])` |

The render API lets you programmatically extract any dashboard component and use it
in notebooks, scripts, reports, or downstream pipelines — without ever opening the Depictio UI.